## Lab3 Goal 1.1 RegNet Solution

Modifies the model used of Goal 1.1 FNN solution to use the ResNet18 pretrained model.

This is a much more complex model than the previous ones. The training time could be long if CUDA accelerators are not available.

In [34]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import models

In [35]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [36]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [37]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model (ResNet18 pretrained model)
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # Load pretrained ResNet18
        self.base_model = models.resnet18(pretrained=True)

        # Adapt input layer for grayscale images (1 channel instead of 3)
        self.base_model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Replace the final fully connected layer
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, num_classes)

    def forward(self, x):
        return self.base_model(x)

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (base_model): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=

In [38]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [39]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [40]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [41]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.885915  [   64/60000]
loss: 1.179215  [ 6464/60000]
loss: 0.743048  [12864/60000]
loss: 0.873996  [19264/60000]
loss: 0.733147  [25664/60000]
loss: 0.753461  [32064/60000]
loss: 0.625169  [38464/60000]
loss: 0.731306  [44864/60000]
loss: 0.617624  [51264/60000]
loss: 0.599669  [57664/60000]
Test Error: 
 Accuracy: 78.8%, Avg loss: 0.578660 

Epoch 2
-------------------------------
loss: 0.434123  [   64/60000]
loss: 0.633843  [ 6464/60000]
loss: 0.381070  [12864/60000]
loss: 0.584573  [19264/60000]
loss: 0.494125  [25664/60000]
loss: 0.595006  [32064/60000]
loss: 0.487962  [38464/60000]
loss: 0.606251  [44864/60000]
loss: 0.494069  [51264/60000]
loss: 0.493701  [57664/60000]
Test Error: 
 Accuracy: 81.6%, Avg loss: 0.499699 

Epoch 3
-------------------------------
loss: 0.319788  [   64/60000]
loss: 0.512359  [ 6464/60000]
loss: 0.304377  [12864/60000]
loss: 0.486701  [19264/60000]
loss: 0.407078  [25664/60000]
loss: 0.493097  [32064/600

In [42]:
torch.save(model.state_dict(), "model_RegNet.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [43]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model_RegNet.pth", weights_only=True))

<All keys matched successfully>

In [44]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[3][0], test_data[3][1]
with torch.no_grad():
    #x = x.to(device)
    x = x.unsqueeze(0).to(device)  # Shape becomes [1, 1, 28, 28]
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Trouser", Actual: "Trouser"
